In [21]:
!pip install transformers
!pip install transformers[torch] # Use pytorch to load weights from hugging face and then to further train
!pip install transformers[sentencepiece]
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: libtorrent 2.0.5-build-libtorrent-rasterbar-qrM5vM-libtorrent-rasterbar-2.0.5-bindings-python has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of libtorrent or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: libtorrent 2.0.5-build-libtorrent-rasterbar-qrM5vM-libtorrent-rasterbar-2.0.5-bindings-python has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of libtorrent or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: libtorrent 2.0.5-build-libtorrent-rasterbar-qrM5vM-libtorrent-rasterbar-2.0.5-bindings-python has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of libtorrent or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: libtorrent 2.0.5-build-libtorrent-rasterbar-qrM5vM-libtorrent-rasterbar-2.0.5-bindings-python has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of libtorrent or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
# Load emotion data set from Hugging Face hub. Emotion is a data set of English Twitter messages with six basic emotions:
# Anger, Fear, Joy , Love, Sadness, Surprise. 
from datasets import load_dataset
emotions = load_dataset("emotion") 

/home/niedag/.local/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
emotions['train'][2]

{'text': 'im grabbing a minute to post i feel greedy wrong', 'label': 3}

In [6]:
# All the features
emotions['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [7]:
# Pre-processing
# Tokenizes (indexes stems of the words - suffixes, prefixes)
from transformers import AutoTokenizer

def tokenize(rows):
    return tokenizer(rows['text'],padding = "max_length", truncation = True)

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

emotions.set_format(type=None)
tokenized_datasets = emotions.map(tokenize, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
# Import the needed libraries from PyTorch and HuggingFace for model training
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    default_data_collator,
    TrainingArguments,
    Trainer
)

2024-04-27 21:10:20.585335: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 21:10:20.777146: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 21:10:20.777191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 21:10:20.809032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-27 21:10:20.880539: I tensorflow/core/platform/cpu_feature_guar

In [22]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [23]:
# Create PyTorch Dataloaders
train_dataloader = DataLoader(
    small_train_dataset,
    sampler=RandomSampler(small_train_dataset),
    batch_size=8,
    collate_fn=default_data_collator,
)

test_dataloader = DataLoader(
    small_test_dataset,
    sampler = SequentialSampler(small_test_dataset),
    batch_size=8,
    collate_fn=default_data_collator,
)

In [24]:
# Instantiate the model

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=6
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    data_collator=default_data_collator,
)

# Fine tune the model with the dataset:
#trainer.train()

/home/niedag/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,1.072800
1000,0.368200
1500,0.308700
2000,0.237400
2500,0.175600
3000,0.165000
3500,0.163700
4000,0.186800
4500,0.145800
5000,0.120300


TrainOutput(global_step=10000, training_loss=0.1863797945022583, metrics={'train_runtime': 1099.6599, 'train_samples_per_second': 72.75, 'train_steps_per_second': 9.094, 'total_flos': 1.059814785024e+16, 'train_loss': 0.1863797945022583, 'epoch': 5.0})

In [37]:
eval_results = trainer.evaluate()

In [38]:
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.35392776131629944, 'eval_runtime': 8.4337, 'eval_samples_per_second': 237.145, 'eval_steps_per_second': 29.643, 'epoch': 5.0}


In [39]:
model.save_pretrained("./fine_tuned_emotions_distilBERT_model")

In [40]:
tokenizer.save_pretrained("./fine_tuned_emotions_distilBERT_model")

('./fine_tuned_emotions_distilBERT_model/tokenizer_config.json',
 './fine_tuned_emotions_distilBERT_model/special_tokens_map.json',
 './fine_tuned_emotions_distilBERT_model/vocab.txt',
 './fine_tuned_emotions_distilBERT_model/added_tokens.json',
 './fine_tuned_emotions_distilBERT_model/tokenizer.json')

In [33]:
print("Parameters/features of the dataset:", emotions.column_names)
print(len(emotions['train']))

Parameters/features of the dataset: {'train': ['text', 'label'], 'validation': ['text', 'label'], 'test': ['text', 'label']}
16000


In [35]:
print(len(emotions['test']))

2000


In [36]:
print(len(emotions['validation']))

2000
